In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# load your model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
model = load_model('/content/drive/MyDrive/Brain_Tumour_Detection/BrainTumour.h5')
print('model loaded')

model loaded


In [5]:
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [6]:
!pip install FPDF

  Preparing metadata (setup.py) ... done
  Created wheel for FPDF: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=ef7feb01254763570c6dda2b650475c4bf225530ca5dd0dd9c8989b753192267
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built FPDF


In [7]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 25.3 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [8]:
import gradio as gr
import tensorflow as tf
import numpy as np
from fpdf import FPDF
from tensorflow.keras.preprocessing import image
from datetime import datetime
import smtplib
from email.message import EmailMessage
import ssl
import os
import shutil
from gtts import gTTS
import uuid

# Load model
model = tf.keras.models.load_model("/content/drive/MyDrive/Brain_Tumour_Detection/BrainTumour.h5")

def sanitize_text(text):
    return text.encode('latin-1', 'replace').decode('latin-1')

def predict_tumor(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    return "Brain Tumor Detected" if prediction[0][0] > 0.5 else "No Brain Tumor"

def generate_pdf_report(name, email, contact, age, gender, result, doctor_list, profile_img_path, output_file="BrainTumorReport.pdf"):
    pdf = FPDF()
    pdf.add_page()
    if os.path.exists(profile_img_path):
        pdf.image(profile_img_path, x=160, y=10, w=30, h=30)

    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, txt=sanitize_text("Brain Tumor Detection Report"), ln=True, align='C')
    pdf.set_font("Arial", size=10)
    pdf.cell(200, 10, txt=sanitize_text(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}"), ln=True, align='C')
    pdf.ln(10)

    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=sanitize_text(f"Patient Name: {name}"), ln=True)
    pdf.cell(200, 10, txt=sanitize_text(f"Email: {email}"), ln=True)
    pdf.cell(200, 10, txt=sanitize_text(f"Contact: {contact}"), ln=True)
    pdf.cell(200, 10, txt=sanitize_text(f"Age: {age}"), ln=True)
    pdf.cell(200, 10, txt=sanitize_text(f"Gender: {gender}"), ln=True)
    pdf.cell(200, 10, txt=sanitize_text(f"Diagnosis Result: {result}"), ln=True)

    pdf.ln(10)
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(200, 10, txt=sanitize_text("Recommended Consultants:"), ln=True)
    pdf.set_font("Arial", size=12)
    for doctor in doctor_list:
        pdf.cell(200, 10, txt=sanitize_text(f"  • {doctor}"), ln=True)

    pdf.ln(10)
    pdf.set_font("Arial", 'I', 10)
    pdf.cell(200, 10, txt=sanitize_text("Note: This is an AI-generated report and not a substitute for clinical diagnosis."), ln=True)
    pdf.output(output_file)
    return output_file

def send_email_with_gmail(recipient_email, subject, body, attachment_file, sender_email, sender_password):
    try:
        msg = EmailMessage()
        msg['Subject'] = subject
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg.set_content(body)

        with open(attachment_file, 'rb') as f:
            msg.add_attachment(f.read(), maintype='application', subtype='pdf', filename=os.path.basename(attachment_file))

        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            server.login(sender_email, sender_password)
            server.send_message(msg)

        return "✅ Email sent successfully!"
    except Exception as e:
        return f"❌ Failed to send email: {str(e)}"

def process_all(name, email, contact, age, gender, profile_img, mri_img, sender_email, sender_password):
    try:
        if not all([name, email, contact, age, gender, profile_img, mri_img, sender_email, sender_password]):
            return "⚠️ Please fill all fields and upload both images.", None

        profile_path = f"profile_{name}.jpg"
        mri_path = f"mri_{name}.jpg"

        # ✅ Use shutil.copy instead of .read() — because profile_img is a filepath
        shutil.copy(profile_img, profile_path)
        shutil.copy(mri_img, mri_path)

        result = predict_tumor(mri_path)
        doctors = ["Dr. Shalini Rao – Apollo Hospitals", "Dr. Vivek Gupta – AIIMS", "Dr. Nisha Roy – Fortis Memorial"]
        pdf_path = generate_pdf_report(name, email, contact, age, gender, result, doctors, profile_path)

        subject = "🧠 Brain Tumor Report"
        body = f"Dear {name},\n\nPlease find your tumor scan report attached.\n\nRegards,\nMedical AI System"

        email_status = send_email_with_gmail(email, subject, body, pdf_path, sender_email, sender_password)
        return f"{result}\n{email_status}", pdf_path

    except Exception as e:
        return f"❌ Error: {str(e)}", None

def chatbot_text_to_audio(user_input):
    if not user_input:
        return "❗ Please ask something.", None

    user_input = user_input.lower()
    if "symptom" in user_input:
        reply = "Symptoms of brain tumors include headaches, seizures, and vision problems."

    elif "what is tumor" in user_input or "define tumor" in user_input:
        reply = "A tumor is an abnormal growth of tissue. In the brain, it can disrupt normal function and cause serious issues."

    elif "benign" in user_input:
       reply = "Benign brain tumors are non-cancerous and usually grow slowly. However, they may still need removal if they press on vital areas."

    elif "malignant" in user_input:
       reply = "Malignant tumors are cancerous and can grow quickly or spread. Immediate medical attention is important."

    elif "urgent" in user_input or "emergency" in user_input:
        reply = "If you're facing severe headaches, seizures, or loss of consciousness, please visit the emergency department immediately."

    elif "support" in user_input or "helpline" in user_input:
        reply = "For emotional support, contact national cancer helplines or speak to a counselor. You're not alone."

    elif "cost" in user_input or "expensive" in user_input:
        reply = "The cost of brain tumor treatment varies. Government hospitals like AIIMS offer affordable care. You can also explore insurance options."

    elif "insurance" in user_input:
        reply = "Many insurance policies cover brain tumor diagnosis and treatment. Contact your provider to confirm the coverage."

    elif "scan" in user_input:
        reply = "MRI (Magnetic Resonance Imaging) is the most common scan for brain tumor detection."

    elif "how to upload" in user_input:
        reply = "You can upload your MRI scan using the upload button in the app. Make sure it’s a clear image."

    elif "how to use" in user_input or "how it works" in user_input:
        reply = "Just upload an MRI image, enter your details, and the system will analyze the image, generate a report, and email it to you."

    elif "safe" in user_input or "secure" in user_input:
        reply = "Yes, your data is secure. This system is built with privacy and safety in mind."

    elif "delete my data" in user_input or "remove info" in user_input:
        reply = "Sure, please mention your email, and your records will be deleted from our database."

    elif "language" in user_input:
        reply = "Currently, I support English. More languages will be added in future updates."

    elif "feedback" in user_input or "suggestion" in user_input:
        reply = "We appreciate your feedback! You can email us or leave your suggestion here."

    elif "location" in user_input or "where am i" in user_input:
        reply = "I can detect nearby hospitals if you share your city name or enable location access."

    elif "risk factors" in user_input:
        reply = "Risk factors for brain tumors include exposure to radiation, family history, and age. But many cases have no known cause."

    elif "stages" in user_input:
        reply = "Brain tumors are staged based on size and spread. A doctor determines the stage through tests and imaging."

    elif "diet" in user_input:
        reply = "A healthy diet during treatment should include fruits, vegetables, and hydration. Consult your doctor for a proper plan."

    elif "thank you so much" in user_input:
        reply = "You're always welcome! 💙 Wishing you good health and peace of mind."

    elif "report" in user_input:
         reply = "If your report says Brain Tumor Detected, consult a neurologist immediately."

    elif "hospital" in user_input:
        reply = "You can visit AIIMS, Apollo, or Fortis hospitals. Tell me your city for nearby suggestions."

    elif "appointment" in user_input:
        reply = "You can book appointments on Apollo247 or Practo."

    elif "treatment" in user_input:
        reply = "Brain tumor treatment options include surgery, radiation therapy, and chemotherapy. Consult a specialist to decide the best plan."

    elif "contact doctor" in user_input or "consult" in user_input:
        reply = "You can consult doctors via telemedicine platforms like Apollo247 or Practo. It's fast and secure."

    elif "pdf" in user_input or "report download" in user_input:
        reply = "Once your MRI is processed, the report PDF will be generated and sent to your email."

    elif "email" in user_input:
        reply = "Reports are sent to your email after prediction. Please check your inbox and spam folder."

    elif "brain tumor" in user_input:
        reply = "A brain tumor is a mass of abnormal cells in your brain. It may be benign or malignant."

    elif "mri" in user_input:
        reply = "An MRI scan is required for brain tumor detection. Please upload your MRI image in the interface."

    elif "nearest hospital" in user_input or "nearby" in user_input:
         reply = "Please tell me your city, and I’ll find the nearest hospitals for you."

    elif "dangerous" in user_input:
         reply = "Some brain tumors can be life-threatening, especially if malignant. Early diagnosis is critical."

    elif "prevention" in user_input:
        reply = "While brain tumors often have no known cause, avoiding radiation exposure and monitoring symptoms can help."

    elif "hello" in user_input or "hi" in user_input:
        reply = "Hi there! 👋 I'm your virtual health assistant. Ask me anything about brain tumor detection or care."

    elif "thank you" in user_input or "thanks" in user_input:
        reply = "You're welcome! 😊 Stay healthy and feel free to ask more questions."

    else:
        reply = "I'm here to assist with brain tumor detection, reports, hospitals, and appointments. Try asking something like 'symptoms' or 'nearest hospital'."

    # Generate audio reply
    tts = gTTS(reply)
    audio_path = f"reply_{uuid.uuid4().hex}.mp3"
    tts.save(audio_path)

    return reply, audio_path

text_audio_chatbot_ui = gr.Interface(
    fn=chatbot_text_to_audio,
    inputs=gr.Textbox(lines=2, placeholder="Type your question here...", label="💬 Ask the Chatbot"),
    outputs=[
        gr.Textbox(label="📝 Text Response"),
        gr.Audio(label="🔊 Audio Response")
    ],
    title="🧠 Brain Tumor Chatbot with Voice Output",
    description="Type your question. You'll get both a text and audio reply."
)

import gradio as gr

with gr.Blocks(title="📅 Book Appointment") as appointment_ui:
    gr.Markdown("""
# 📅 Book a Doctor Appointment

Easily book appointments with trusted medical platforms:

🔗 [Apollo 24/7](https://www.apollo247.com/)

🔗 [Practo](https://www.practo.com/)

🔗 [AIIMS - Government Portal](https://ors.gov.in/index.html)

🔗 [Fortis Healthcare](https://www.fortishealthcare.com/)

---

💡 *Tip: Visit the site based on your location and hospital preference.*
    """)


# Gradio UI
inputs = [
    gr.Textbox(label="🧑 Patient Name"),
    gr.Textbox(label="📧 Email"),
    gr.Textbox(label="📱 Contact Number"),
    gr.Textbox(label="🎂 Age"),
    gr.Radio(["Male", "Female", "Other"], label="⚧ Gender"),
    gr.File(label="🖼️ Upload Profile Photo"),
    gr.File(label="🧠 Upload MRI Scan"),
    gr.Textbox(label="📨 Your Gmail Address"),
    gr.Textbox(label="🔑 Gmail App Password", type="password"),
]

outputs = [
    gr.Textbox(label="📄 Diagnosis & Email Status"),
    gr.File(label="📎 Download Report PDF"),
]

gr.Interface(
    fn=process_all,
    inputs=inputs,
    outputs=outputs,
    title="🧠 Brain Tumor Detection & Email Report System",
    description="Upload patient info, MRI, and get a full diagnostic report emailed with PDF."
)

gr.TabbedInterface(
    interface_list=[
        gr.Interface(fn=process_all, inputs=inputs, outputs=outputs,
                     title="🧠 Brain Tumor Detection & Email Report System",
                     description="Upload MRI & patient info to generate and email a diagnostic report."),
        text_audio_chatbot_ui,
        appointment_ui
    ],
    tab_names=[
        "📄 Tumor Detector",
        "💬 Chatbot (Text + Voice)",
        "📅 Book Appointment"
    ]
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7c3087d3d5b9e6107a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
